# CustomLayer 定義

如果你想要自己根據論文公式刻出一個model，下面一有一個最基本的範例可供使用
PyTorch 在定義model會需要用到Parameter來打包可訓練參數
這樣在優化器指定要優化的參數時，才可以透過Module.parameter()這個函式來快速調參

而要注意的是當資料是以Batch的形式呈現，計算時要注意batch的軸不能消失
否則在訓練資料的過程中就會失去Batch Training的意義

也就是說公式需要改寫為
## $$y=xW^T+b$$

In [1]:
import torch

c:\Users\Wen2Tee5\Desktop\Postgraduate\torchPlayGround\torchEnv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

class LinearLayer(torch.nn.Module):

    def __init__(self, in_feature, out_feature) -> None:
        super(LinearLayer, self).__init__()
        self.w1 = torch.nn.Parameter(torch.randn(size = (out_feature, in_feature), requires_grad=True))
        self.b1 = torch.nn.Parameter(torch.randn(size = (out_feature, )))

    def forward(self, x):
        x = x @ self.w1.T + self.b1
        return x

In [3]:
linear = LinearLayer(20, 10)

In [4]:
for parm in linear.parameters():
    print(parm.shape)

torch.Size([10, 20])
torch.Size([10])


In [5]:
x = torch.randn(size=(20,20))

In [6]:
temp = linear(x)

In [7]:
temp.shape

torch.Size([20, 10])

In [8]:
tempModel = torch.nn.Linear(20,10)

In [9]:
for parm in tempModel.parameters():
    print(parm.shape)

torch.Size([10, 20])
torch.Size([10])


In [10]:
x = torch.randn(size=(2, 10))
b = torch.randn(size=(10,))

In [11]:
print(x)
print(b)

tensor([[ 1.3411,  0.6916, -0.7753,  1.3080,  0.5635,  0.1124, -0.5848, -1.4440,
         -1.8189,  0.7487],
        [-1.0637,  0.8090, -0.5334,  1.1508, -1.7645,  0.2689,  0.5493, -0.4905,
          1.6187,  1.2298]])
tensor([ 0.5867,  0.5211, -0.6068, -1.1766, -1.5682, -2.0363,  0.3843, -1.9030,
         2.3666, -0.2004])


In [12]:
print(x+1)

tensor([[ 2.3411,  1.6916,  0.2247,  2.3080,  1.5635,  1.1124,  0.4152, -0.4440,
         -0.8189,  1.7487],
        [-0.0637,  1.8090,  0.4666,  2.1508, -0.7645,  1.2689,  1.5493,  0.5095,
          2.6187,  2.2298]])
